In [ ]:
!pip install duckdb

In [ ]:
import duckdb

In [ ]:
# !rm /p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/*.parquet
!ls /p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet

In [ ]:
duckdb.query('''
    select count(*)
    from '/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/0_0.parquet'
''').to_df()

In [ ]:
import datafusion
from datafusion import column
from datafusion import functions as f

ctx = datafusion.SessionContext()
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet/*.parquet")
ctx

In [ ]:
df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet/*.parquet")
df

In [ ]:
%%time
results = df.aggregate([column("filename")], [f.sum(column("duration"))]).sort(column("SUM(?table?.duration)").sort(ascending=False)).collect()
len(results)

In [ ]:
%%time
sum([len(results[r].columns[0]) for r in range(len(results))])

In [ ]:
%%time
results[0].column(0)[0], results[0].column(1)[0]

In [ ]:
%%time
results = df.aggregate([column("filename")], [f.sum(column("size"))]).sort(column("SUM(?table?.size)").sort(ascending=False)).collect()
len(results)

In [ ]:
%%time
sum([len(results[r].columns[0]) for r in range(len(results))])

In [ ]:
%%time
results[0].column(0)[0], results[0].column(1)[0]

In [ ]:
%%time
results = df.aggregate([column("filename"), column("proc_id"), column("io_cat")], [f.sum(column("size"))]).sort(column("SUM(?table?.size)").sort(ascending=False)).collect()
len(results)

In [ ]:
%%time
sum([len(results[r].columns[0]) for r in range(len(results))])

In [ ]:
%%time
results[0].column(0)[0], results[0].column(1)[0], results[0].column(2)[0], results[0].column(3)[0]

In [ ]:
results[0].column(0)[1], results[0].column(1)[1], results[0].column(2)[1], results[0].column(3)[1]

In [ ]:
ctx2 = datafusion.SessionContext()
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet/*.parquet")
ctx2.register_parquet("montage", "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet/*.parquet")

ctx2

In [ ]:
%%time
sql_df = ctx2.sql("SELECT filename, SUM(duration) FROM montage GROUP BY filename LIMIT 100")
sql_res = sql_df.collect()
sql_res

In [ ]:
len(sql_res)

In [ ]:
sql_res[0].column(0)[0], sql_res[0].column(1)[0]

In [ ]:
ctx2.register_parquet("hacc", "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet/*.parquet")

In [ ]:
%%time
sql_hacc_df = ctx2.sql("SELECT filename, SUM(duration) FROM hacc GROUP BY filename LIMIT 100")
sql_hacc_res = sql_hacc_df.collect()
sql_hacc_res

In [ ]:
sql_hacc_res[0].column(0)[0], sql_hacc_res[0].column(1)[0]

In [ ]:
ctx2.register_parquet("genome", "/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/*.parquet")

In [ ]:
%%time
sql_gen_df = ctx2.sql("SELECT filename, SUM(duration) FROM genome GROUP BY filename LIMIT 100")
sql_gen_res = sql_gen_df.collect()
sql_gen_res

In [ ]:
gen_df_0 = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/0_0.parquet")
gen_df_0

In [ ]:
%%time
gen_res_0 = gen_df_0.aggregate(
    [column("filename"), column("proc_id"), column("io_cat")], 
    [f.sum(column("duration")), f.sum(column("size")), f.count(column("index"))]
).sort(column("COUNT(?table?.index)").sort(ascending=False)).collect()
len(gen_res_0)

In [ ]:
gen_res_0[0].column(0)[0], gen_res_0[0].column(1)[0], gen_res_0[0].column(2)[0], gen_res_0[0].column(3)[0], gen_res_0[0].column(4)[0], gen_res_0[0].column(5)[0]

In [ ]:
gen_res_0[0].to_pandas()

In [ ]:
cm1_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"

cm1_ddf = dd.read_parquet(f"{cm1_dir}/*.parquet")

cm1_ddf

In [ ]:
import polars as pl

# cm1_pl_df = pl.read_parquet(f"{cm1_dir}/*.parquet")

# cm1_pl_df

In [ ]:
%%time
cm1_pl_agg_df = cm1_pl_df \
    .groupby(["filename", "proc_id", "io_cat"]) \
    .agg([pl.sum("duration"), pl.sum("size")]) \
    .sort("duration", reverse=True)
cm1_pl_agg_df


In [ ]:
cm1_pl_agg_df[0]['filename']

In [ ]:
gen_dir = "/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
# gen_pl_df = pl.read_parquet(f"{gen_dir}/*.parquet", n_rows=10)

In [ ]:
q1 = (
    pl.scan_parquet(f"{gen_dir}/*.parquet")
    .groupby(["file_id", "proc_id", "io_cat"])
    .agg([pl.sum("duration"), pl.sum("size")])
    .sort("duration", reverse=True)
)
# q1

In [ ]:
%%time
q1.collect()